In [3]:
import pandas as pd
from py_midicsv import midicsv
from os import walk
from tqdm import tqdm
import numpy as np
import tensorflow as tf

In [12]:
# Read-in
path = r'D:\Documents\GitHub\Pyotr\MIDI Files\cho\prelud7.mid'

# Name the columns
df = pd.DataFrame([line.split(", ") for line in midicsv.parse(path)])
columns = ['Track', 'Time', 'Type']
for i in range(3, len(df.columns)):
    columns.append(f'TypeInfo{i}')


df.columns = columns
df.replace('\n', '', regex=True, inplace=True)

In [63]:
# Get Note Data (Right Hand)
note_start = df[df.Type.str.contains('Note_on')].sort_values('Time', ascending=True)
RH = (note_start[['Time', 'Track', 'TypeInfo4']].groupby(['Track', 'Time'], as_index=False).agg(list)
                .pivot(index='Track', columns='Time', values='TypeInfo4')
           [[str(x) for x in note_start.Time.astype(int).sort_values(ascending=True).unique()]].transpose())['2']
RH

Time
480                      [64]
720                      [73]
900                      [74]
960              [62, 68, 71]
1200             [71, 68, 62]
1440             [62, 68, 71]
1920                 [78, 74]
2160                 [72, 75]
2340                 [73, 76]
2400                 [73, 81]
2640                 [73, 81]
2880                 [81, 73]
3360                 [73, 64]
3600                 [61, 70]
3780                 [62, 71]
3840                 [66, 74]
4080                 [66, 74]
4320                 [66, 74]
4800                 [68, 62]
5040                 [68, 62]
5220                 [69, 61]
5280                     [73]
5520                 [61, 73]
5760                 [73, 61]
6240                     [64]
6480                 [73, 68]
6660                     [74]
6720             [71, 68, 62]
6960             [62, 68, 71]
7200             [68, 62, 71]
7680                 [78, 74]
7920                 [72, 75]
8100                 [76, 73]
8160 

In [40]:
train_df = pd.DataFrame(index=[x for x in range(0, len(RH)-cuts)], columns=['SQ_'+str(x)for x in range(0, cuts)]+['target'])

In [59]:
# Split into trainable chunks
cuts = 4
input_cols = ['SQ_'+str(x)for x in range(0, cuts)]
train_df = pd.DataFrame(index=[x for x in range(0, len(RH)-cuts)], columns=input_cols+['target'])
for i in range(0, len(RH)-cuts):
    train_df.loc[i, input_cols] = RH[i:i+cuts].tolist()
    train_df.loc[i, 'target'] = RH[i+cuts]

In [64]:
train_df

,SQ_0,SQ_1,SQ_2,SQ_3,target
0,[64],[73],[74],"[62, 68, 71]","[71, 68, 62]"
1,[73],[74],"[62, 68, 71]","[71, 68, 62]","[62, 68, 71]"
2,[74],"[62, 68, 71]","[71, 68, 62]","[62, 68, 71]","[78, 74]"
3,"[62, 68, 71]","[71, 68, 62]","[62, 68, 71]","[78, 74]","[72, 75]"
4,"[71, 68, 62]","[62, 68, 71]","[78, 74]","[72, 75]","[73, 76]"
5,"[62, 68, 71]","[78, 74]","[72, 75]","[73, 76]","[73, 81]"
6,"[78, 74]","[72, 75]","[73, 76]","[73, 81]","[73, 81]"
7,"[72, 75]","[73, 76]","[73, 81]","[73, 81]","[81, 73]"
8,"[73, 76]","[73, 81]","[73, 81]","[81, 73]","[73, 64]"
9,"[73, 81]","[73, 81]","[81, 73]","[73, 64]","[61, 70]"
